## Importing of Libraries

In [1]:
import time
from copy import deepcopy

import numpy as np
import torch
from dlordinal.layers import StickBreakingLayer
from dlordinal.losses import TriangularCrossEntropyLoss
from dlordinal.datasets import FGNet
from sklearn.metrics import (accuracy_score, cohen_kappa_score,
                             confusion_matrix, mean_absolute_error)
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from torch import cuda
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor
from tqdm import tqdm

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the *DataLoader*, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [2]:
optimiser_params = {
    'lr': 1e-3,
    'bs': 200,
    'epochs': 5,
    's': 2,
    'c': 0.2,
    'beta': 0.5
}

workers = 3

Now we use the *FGNet* method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the *StratifiedShuffleSplit* method. Finally, with all the partitions, we load the images using a method called *DataLoader*.

In [3]:
fgnet = FGNet(root="./datasets/fgnet", download=True, process_data=True)

complete_train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

num_classes = len(complete_train_data.classes)
classes = complete_train_data.classes
targets = complete_train_data.targets

# Create a validation split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
sss_splits = list(
    sss.split(X=np.zeros(len(complete_train_data)), y=complete_train_data.targets)
)
train_idx, val_idx = sss_splits[0]

# Create subsets for training and validation
train_data = Subset(complete_train_data, train_idx)
val_data = Subset(complete_train_data, val_idx)

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Create dataloaders
train_dataloader = DataLoader(
    train_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
val_dataloader = DataLoader(
    val_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
test_dataloader = DataLoader(
    test_data, batch_size=optimiser_params["bs"], shuffle=False, num_workers=workers
)

# Get image shape
img_shape = None
for X, _ in train_dataloader:
    img_shape = list(X.shape[1:])
    break
print(f"Detected image shape: {img_shape}")

# Define class weights for imbalanced datasets
classes_array = np.array([int(c) for c in classes])

class_weights = class_weight.compute_class_weight(
    "balanced", classes=classes_array, y=targets
)
print(f"{class_weights=}")
class_weights = (
    torch.from_numpy(class_weights).float().to(device)
)  # Transform to Tensor

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cuda device
Detected image shape: [3, 128, 128]
class_weights=array([1.60843373, 0.55394191, 1.02692308, 0.78070175, 1.12184874,
       2.34210526])


## Model

We are using a pretrained *ResNet* model, which has previously been trained on ImageNet. We are modifying the last fully connected layer by a methodology called *Stick Breaking*.

The stick breaking approach considers the problem of breaking a stick of length 1 into J segments. This methodology is related to non-parametric Bayesian methods and can be considered a subset of the random allocation processes [1].

Finally, we define the *Adam* optimiser, which is used to adjust the network's weights and minimize the error of a loss function.

[1]: Vargas, Víctor Manuel et al. (2022). *Unimodal regularisation based on beta distribution for deep ordinal regression.* Pattern Recognition, 122, 108310. Elsevier. doi.org/10.1016/j.patcog.2021.108310

In [4]:
model = models.resnet18(pretrained=True)
model.fc = StickBreakingLayer(model.fc.in_features, num_classes)
model = model.to(device)

# Optimizer and scheduler
optimizer = Adam(model.parameters(), lr=optimiser_params["lr"])

## Loss Function

$$
\begin{align*}
f(x; a_j, b_j, c_j) &= \begin{cases}
   0, & x < a_j, \\
   \frac{2(x - a_j)}{(b_j - a_j)(c_j - a_j)}, & a_j \leq x < c_j, \\
   \frac{2(b_j - x)}{(b_j - a_j)(b_j - c_j)}, & c_j \leq x < b_j, \\
   0, & b_j \leq x,
\end{cases}
\end{align*}
$$


The triangular distribution [1] can be determined using three parameters a, b and c, which define the lower limit, upper limit, and mode, respectively. These parameters also determine the x coordinate of each of the vertices of the triangle.

The distributions employed for the extreme classes should differ from those utilized for the intermediate ones. Consequently, the distributions for the initial and final classes should allocate their probabilities just in one direction: positively for the first class and negatively for the last one.

[1]: Víctor Manuel Vargas, Pedro Antonio Gutiérrez, Javier Barbero-Gómez, and César Hervás-Martínez (2023). *Soft Labelling Based on Triangular Distributions for Ordinal Classification.* Information Fusion, 93, 258--267. doi.org/10.1016/j.inffus.2023.01.003


In [5]:
loss_fn = TriangularCrossEntropyLoss(num_classes=num_classes).to(device)

## Metrics

In [6]:
# Metrics computation


def compute_metrics(y_true: np.ndarray, 
    y_pred: np.ndarray, 
    num_classes: int):

    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)

    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    labels = range(0, num_classes)

    # Metrics calculation
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic', labels=labels)
    ms = minimum_sensitivity(y_true, y_pred, labels=labels)
    mae = mean_absolute_error(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    off1 = accuracy_off1(y_true, y_pred, labels=labels)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

    metrics = {
        'QWK': qwk,
        'MS': ms,
        'MAE': mae,
        'CCR': acc,
        '1-off': off1,
        'Confusion matrix': conf_mat
    }

    return metrics


def _compute_sensitivities(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

	sum = np.sum(conf_mat, axis=1)
	mask = np.eye(conf_mat.shape[0], conf_mat.shape[1])
	correct = np.sum(conf_mat * mask, axis=1)
	sensitivities = correct / sum

	sensitivities = sensitivities[~np.isnan(sensitivities)]

	return sensitivities


def minimum_sensitivity(y_true, y_pred, labels=None):
	return np.min(_compute_sensitivities(y_true, y_pred, labels=labels))


def accuracy_off1(y_true, y_pred, labels=None):
	if len(y_true.shape) > 1:
		y_true = np.argmax(y_true, axis=1)
	if len(y_pred.shape) > 1:
		y_pred = np.argmax(y_pred, axis=1)

	conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
	n = conf_mat.shape[0]
	mask = np.eye(n, n) + np.eye(n, n, k=1), + np.eye(n, n, k=-1)
	correct = mask * conf_mat

	return 1.0 * np.sum(correct) / np.sum(conf_mat)


def print_metrics(metrics):
    print("")
    print('Confusion matrix :\n{}'.format(metrics['Confusion matrix']))
    print("")
    print('MS: {:.4f}'.format(metrics['MS']))
    print("")
    print('QWK: {:.4f}'.format(metrics['QWK']))
    print("")
    print('MAE: {:.4f}'.format(metrics['MAE']))
    print("")
    print('CCR: {:.4f}'.format(metrics['CCR']))
    print("")
    print('1-off: {:.4f}'.format(metrics['1-off']))

## Training Process

In [7]:
def train(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    H: dict,
    num_classes: int,
):  # H: dict
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Train progress")
    model.train()
    mean_loss, accuracy = 0, 0
    y_pred, y_true = None, None

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Inputs and labels to device

        # Compute prediction error and accuracy of the training process
        pred = model(X)
        loss = loss_fn(pred, y)

        mean_loss += loss
        accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Stack predictions and true labels to determine the confusion matrix
        pred_np = pred.argmax(1).cpu().detach().numpy()
        true_np = y.cpu().detach().numpy()
        if y_pred is None:
            y_pred = pred_np
        else:
            y_pred = np.concatenate((y_pred, pred_np))

        if y_true is None:
            y_true = true_np
        else:
            y_true = np.concatenate((y_true, true_np))

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy)
        progress_bar.update(1)

    accuracy /= size
    mean_loss /= num_batches

    H["train_loss"].append(loss.cpu().detach().numpy())
    H["train_acc"].append(accuracy)

    # Confusion matrix for training
    labels = range(0, num_classes)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    print("")
    print("Train Confusion matrix :\n{}".format(conf_mat))
    print("")

    return accuracy, mean_loss

## Test Process

In [8]:
def test(
    test_dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: torch.device,
    num_classes: int,
):
    num_batches = len(test_dataloader)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Test progress")
    model.eval()
    test_loss = 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)  # inputs and labels to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # Stack predictions and true labels
            pred_np = pred.argmax(1).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

            # Update progress bar
            progress_bar.set_postfix(loss=test_loss / (batch + 1))
            progress_bar.update(1)

    test_loss /= num_batches
    metrics = compute_metrics(y_true, y_pred, num_classes)
    print_metrics(metrics)

    return metrics, test_loss

## Validation Process

In [9]:
def validate(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    loss_fn: torch.nn.Module,
    device: torch.device,
    H: dict,
    num_classes: int,
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, accuracy = 0, 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss += loss_fn(pred, y)
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

            pred_np = pred.argmax(1).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

    accuracy /= size
    loss /= num_batches

    H["val_loss"].append(loss.cpu().detach().numpy())
    H["val_acc"].append(accuracy)

    metrics = compute_metrics(y_true, y_pred, num_classes)

    return metrics, accuracy, loss

## Results

In [10]:
H = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# To store validation metrics
validation_metrics = {}

# Definition to store best model weights
best_model_weights = model.state_dict()
best_qwk = 0.0

# Start time
start_time = time.time()

for e in range(optimiser_params["epochs"]):
    train_acc, train_loss = train(
        train_dataloader, model, loss_fn, optimizer, device, H, num_classes=num_classes
    )
    validation_metrics, val_acc, val_loss = validate(
        val_dataloader, model, loss_fn, device, H, num_classes=num_classes
    )

    if validation_metrics["QWK"] >= best_qwk:
        best_qwk = validation_metrics["QWK"]
        best_model_weights = deepcopy(model.state_dict())

    print("[INFO] EPOCH: {}/{}".format(e + 1, optimiser_params["epochs"]))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(train_loss, train_acc))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(val_loss, val_acc))

# Store last train error
train_error = H["train_loss"][-1]

# Restore best weights
model.load_state_dict(best_model_weights)

# Start evaluation
print("[INFO] Network evaluation ...")

test_metrics, test_loss = test(
    test_dataloader, model, loss_fn, device, num_classes=num_classes
)

# End time
end_time = time.time()
print("\n[INFO] Total training time: {:.2f}s".format(end_time - start_time))

Train progress:  25%|██████▎                  | 1/4 [00:01<00:05,  1.85s/it, accuracy=57, loss=1.89]

tensor([[-1.4524, -2.3870, -3.8288, -3.5049, -4.5601, -6.4897],
        [-1.8007, -3.1167, -3.4451, -3.7290, -4.3031, -5.5605],
        [-1.5087, -1.7912, -2.5809, -3.0644, -3.0782, -3.8081],
        ...,
        [-1.4835, -2.4432, -3.3595, -3.5126, -4.2919, -5.4658],
        [-1.4676, -2.1262, -3.5059, -4.6662, -5.5921, -6.3200],
        [-1.7479, -2.2322, -3.6313, -4.1351, -4.0532, -5.1453]],
       device='cuda:0', grad_fn=<LogBackward0>)
tensor([3, 3, 4, 1, 4, 3, 0, 1, 0, 1, 1, 2, 5, 2, 4, 1, 3, 1, 1, 2, 3, 1, 4, 1,
        3, 3, 2, 3, 3, 1, 0, 1, 0, 3, 4, 4, 1, 0, 1, 1, 3, 2, 0, 4, 3, 3, 4, 5,
        4, 1, 2, 3, 0, 4, 1, 1, 3, 4, 1, 3, 0, 2, 1, 1, 0, 0, 4, 4, 2, 1, 2, 4,
        5, 4, 2, 4, 1, 4, 2, 4, 1, 2, 1, 1, 1, 0, 3, 3, 0, 1, 2, 1, 3, 3, 1, 0,
        3, 3, 1, 0, 4, 1, 3, 4, 2, 0, 3, 2, 2, 1, 3, 2, 3, 4, 3, 3, 3, 5, 3, 1,
        3, 3, 3, 1, 3, 4, 4, 0, 3, 2, 0, 3, 3, 1, 2, 1, 5, 3, 3, 4, 5, 2, 1, 1,
        3, 2, 1, 3, 4, 0, 3, 3, 5, 0, 0, 1, 3, 1, 1, 1, 5, 1, 3, 0, 3, 1, 

Train progress:  75%|██████████████████▊      | 3/4 [00:02<00:00,  1.82it/s, accuracy=125, loss=1.6]

tensor([[-1.6488, -2.5838, -2.8312, -1.9495, -2.4759, -3.4682],
        [-1.7872, -3.9560, -6.2586, -8.3967, -7.9779, -8.0079],
        [-2.4078, -2.5133, -2.5584, -1.7485, -2.3675, -2.6099],
        ...,
        [-2.0260, -2.0912, -2.4092, -2.4302, -2.1441, -2.6813],
        [-1.9830, -3.4873, -5.6127, -4.9098, -5.1187, -5.4987],
        [-1.4839, -3.4648, -3.5702, -3.6756, -2.7831, -4.2399]],
       device='cuda:0', grad_fn=<LogBackward0>)
tensor([4, 0, 5, 1, 2, 2, 5, 1, 4, 1, 2, 3, 1, 0, 3, 1, 1, 3, 4, 3, 3, 4, 3, 1,
        1, 2, 2, 1, 4, 4, 2, 0, 1, 4, 1, 4, 1, 4, 1, 4, 3, 4, 1, 4, 1, 5, 2, 0,
        3, 1, 4, 2, 2, 2, 4, 3, 4, 1, 3, 3, 1, 1, 1, 1, 1, 0, 2, 2, 5, 3, 4, 1,
        4, 4, 4, 3, 3, 1, 3, 4, 5, 1, 1, 3, 5, 1, 1, 1, 1, 5, 1, 2, 0, 3, 5, 2,
        3, 2, 4, 1, 3, 1, 1, 1, 4, 4, 1, 1, 2, 1, 1, 0, 0, 3, 1, 4, 3, 3, 1, 2,
        1, 4, 2, 5, 4, 4, 4, 3, 1, 0, 5, 3, 5, 5, 2, 3, 4, 5, 1, 0, 5, 1, 1, 0,
        0, 4, 3, 1, 2, 1, 0, 3, 5, 4, 2, 1, 1, 2, 3, 4, 0, 1, 1, 2, 3, 3, 

Train progress: 100%|█████████████████████████| 4/4 [00:02<00:00,  1.78it/s, accuracy=125, loss=1.6]

...

Train progress: 100%|████████████████████████| 4/4 [00:01<00:00,  3.14it/s, accuracy=525, loss=1.13]

tensor([[-2.4079, -2.5133, -2.3843, -2.7590, -5.0616, -7.3641],
        [-2.4079, -1.4922, -3.2309, -5.5335, -5.2030, -6.4739],
        [-2.4079, -1.5695, -2.9646, -5.2672, -4.4221, -5.5711],
        [-2.4079, -1.4918, -3.1957, -5.4983, -5.6037, -7.3999],
        [-2.4079, -2.5133, -1.7193, -2.1326, -4.1510, -6.4536],
        [-2.4079, -3.6930, -5.3418, -5.4472, -4.6986, -6.0549],
        [-2.4079, -2.5133, -1.9900, -1.8425, -3.6143, -5.9169],
        [-2.4079, -2.5133, -2.6187, -2.6963, -1.8397, -2.6284],
        [-2.4079, -1.4967, -3.2803, -4.7091, -5.9607, -7.2425],
        [-2.4079, -2.5133, -2.3606, -4.0126, -6.1848, -8.0438],
        [-2.4079, -2.5133, -2.6187, -1.9447, -2.0538, -3.6498],
        [-2.4079, -2.5133, -2.6187, -2.2005, -2.1588, -4.2106],
        [-2.4079, -1.5362, -3.0169, -4.3081, -5.9160, -7.2833],
        [-2.4079, -2.5133, -2.6187, -2.2347, -2.6431, -4.8954],
        [-2.4079, -2.5133, -2.6187, -2.7240, -2.5862, -2.0420],
        [-2.4079, -2.5133, -2.6187, -1.8

[INFO] EPOCH: 5/5
Train loss: 1.106344, Train accuracy: 0.7721
Val loss: 1.767812, Val accuracy: 0.4132

[INFO] Network evaluation ...


Test progress: 100%|███████████████████████████████████████| 2/2 [00:00<00:00,  2.56it/s, loss=3.22]


Confusion matrix :
[[24  1  0  1  0  0]
 [29 25  5  2  0  0]
 [ 3  7 14 10  0  0]
 [ 5  5 10 13  5  1]
 [ 2  4 10  8  2  3]
 [ 2  0  2  2  1  5]]

MS: 0.0690

QWK: 0.5978

MAE: 0.8905

CCR: 0.4129

1-off: 0.8060

[INFO] Total training time: 10.62s
